# 8) Hyperbolic Equations

Last time:

- Measuring errors  
- Stability  
- Consistency  
- Second order derivatives  
- Matrix representations and properties  
- Second order derivative with Dirichelet boundary conditions  
- Discrete Green’s functions  
- Interpolation by Vandermonde matrices  

Today:
1. Introduction to Hyperbolic PDEs  
  1.1. The wave equation  
  1.2 A more general hyperbolic equation  
2. System of Hyperbolic Equations  
3. Hyperbolic Equations with Variable Coefficients  
  3.1 Hyperbolic Systems with Variable Coefficients  
4. Boundary Conditions  
5. FD schemes for hyperbolic equations  
  5.1 FD schemes for the 1D wave equation  
  5.2 Example using Leapfrog  
  5.3 Example using Lax-Friedrichs  
6. Stability of hyperbolic PDEs  
  6.1 The Courant-Friedrichs-Lewy Condition  
7. Conditional and unconditional stability

## 1. Introduction to Hyperbolic PDEs 

We begin with an overview of Hyperbolic PDEs; from the simplest model equation, to hyperbolic systems, and equations with variable coefficients.

We have already introduced central concepts in Numerical Analysis, such as convergence, consistency, and stability for Finite Difference (FD) schemes. 

We saw how these concepts are related by the Lax Equivalence Theprem.

### 1.1 The wave equation

The full second-order two-way wave equation yields solutions propagating both ways; it describes a standing wavefield resulting from superposition of two waves in opposite directions. By formally "factoring" the differential operator
  
$$
  \left( \partial_t^2-a^2\partial_x^2 \right) u =
  \left( \partial_t-a\partial_x \right)
  \left( \partial_t+a\partial_x \right) u 
  \equiv
  \left( \partial_t+a\partial_x \right)
  \left( \partial_t-a\partial_x \right) u 
  = 0,
$$
  
it is clear that solutions to either

$$
  \left( \partial_t-a\partial_x \right) u = 0, \quad\hbox{or}\quad
  \left( \partial_t+a\partial_x \right) u = 0,
$$

are solutions to the original equation.

These are known as **one-way wave equations** or **transport equations** or **advection equations** describing a physical transport mechanism (with propagation speed $a$ [Units of Length /Units of time]).

The simplest prototype for Hyperbolic PDEs is the **one-way wave equation**

$$
  u_t(t,x) + a u_x(t,x) =0 \tag{8.1},
$$

where $a$ is a constant, $t\in\mathbb{R}^{+}$ represents time, and $x\in\mathbb{R}$ the spatial location.  The initial state, $u(0,x)$, must be specified.

Once the **initial condition (IC)** $u(0,x)=u_0(x)$ is specified, the **unique solution** to the one-way wave equation for $t>0$ is given by
  
$$ 
  u(t,x) = u_0(x-at).
$$

The solution at time $t$ is just a shift of the initial value, $u_0(x)$. 

- When $a>0$ it is a shift to the _right_ and when $a<0$ it is
  a shift to the _left_.

The solution depends only on the value of $\xi=x-at$.

These are lines in the $(t,x)$-plane that are called **characteristic lines** or **characteristics**, and

$$
\textrm{units}(a) = \frac{ \textrm{units}(x)}{ \textrm{units}(t)} = \frac{[\textrm{length} ]}{[ \textrm{time}]},
$$
  
hence $a$ is the **propagation speed**.


This is typical for Hyperbolic Equations: 

:::{note}
- **The solution propagates with finite speed along characteristics.**
:::


We note that the exact solution

$$
  u(t,x) = u_0(x-at),
$$

requires no differentiability of $u$ (or $u_0$), whereas the equation

$$
  u_t + a u_x =0,
$$

appears to only make sense if $u$ is differentiable.




![Condensation near an aircraft shows evidence of a transonic expansion fan](../img/transonic_expansion_fan.jpg)
This picture shows a volume with low pressure near the rear of the aircraft at high subsonic airspeeds. [Source: [Transonic speed regime Wiki page](http://en.wikipedia.org/wiki/Transonic_speed)]. 

Hyperbolic equations feature solutions that are discontinuous (worse than non-differentiable); e.g., the **sonic boom** produced by an aircraft exceeding the speed of sound (Mach-1, or $\approx$ 750 miles per hour at sea level) is an example of this phenomena.  

The discontinuity creates a **shock wave**.

![Merging Shockwaves](../img/nasa_merging_shockwaves.jpg)
[Source: [Phys.org](https://phys.org/news/2019-03-nasa-captures-unprecedented-images-supersonic.html)]

- Devising numerical schemes which allow for discontinuous solutions requires "a bit" of ingenuity.



## 1.2 A more general hyperbolic equation

\begin{align*}
    u_t + a u_x + b u &= f(t,x),\quad t>0 \\
    u(0,x) &= u_0(x)
\end{align*}

Where $a$ and $b$ are constants. 

We can introduce the following change of variables (and its inverse):

$$
  \left\{\begin{array}{rcl}\tau&=&t\\\xi &=&x-at,\end{array}\right.
  \quad\quad
  \left\{\begin{array}{rcl}t&=&\tau\\x &=&\xi+a\tau\end{array}\right.
$$

With $\tilde{u}(\tau,\xi)=u(t,x)$, we can transform the PDE to an ODE along the characteristics:

$$
\tilde{u}_{\tau} = -b\tilde{u} + f(\tau,\xi+a\tau).
$$

The exact solution is given by

$$
  \tilde{u}(\tau,\xi) = u_0(\xi)e^{-b\tau} + \int_0^{\tau} f(\sigma, \xi
  + a \sigma)e^{-b(\tau-\sigma)}\,d\sigma,
$$

which expressed in the original variables is
$$
  u(t,x) = u_0(x-at)e^{-bt} + \int_{0}^{t} f(s,x-a(t-s))e^{-b(t-s)}\,ds.
$$


With some work this method can be extended to nonlinear equations of the form

$$
  u_t + u_x = f(t,x,u),\quad
  \textbf{Note:}\ \hbox{$f$ depends on $u$}
$$

From a numerical point of view, the key thing to note is that the solution evolves with **finite speed along the characteristics**.

## 2. System of Hyperbolic Equations

Now consider systems of hyperbolic equations with constant coefficients in one space dimension; ${\mathbf{u}}$ is now a $d$-dimensional vector (containing various quantities, e.g., density ($\rho$), pressure ($p$), velocity ($v$), energy ($E$), and linear momentum ($\rho v$) of a fluid or gas).

:::{admonition} Definition: **Hyperbolic System**
A system of the form

$$
    {\mathbf{u}}_t + A {\mathbf{u}}_x + B {\mathbf{u}} = F(t,x)
$$ 
is hyperbolic if the matrix $A$ is diagonalizable with real eigenvalues.
:::

The matrix $A$ is **diagonalizable**, if there exists a non-singular matrix $P$ such that

$$
  PAP^{-1} = \textrm{diag}(\lambda_1,\dots,\lambda_d) = \Lambda,
$$
is a diagonal matrix.  

The eigenvalues $\lambda_1,\dots,\lambda_d$ are the **characteristic speeds** of the system.

In the easiest case, $B=0$, we get

$$
  \mathbf{w}_t + \Lambda \mathbf{w}_x = P F(t,x) = \tilde{F}(t,x)
$$

under the change of variables $\mathbf{w} = P \mathbf{u}$.  This is a reduction to $d$ independent scalar hyperbolic equations.

When $B\not=0$, the resulting system is coupled, but only in undifferentiated terms.  The lower order term $B\mathbf{u}$ causes growth, decay, or oscillations in the solution but **does not** alter the primary feature of solutions propagating along characteristics.

### Example 8.1:

$$
\left[\begin{array}{c}u\\v\end{array}\right]_{t}
+
\left[\begin{array}{cc}2&1\\1&2\end{array}\right]
\left[\begin{array}{c}u\\v\end{array}\right]_x
= 0
$$
with $u(0,x)\equiv u_0=1$ if $|x|\le1$, and 0 otherwise; and $v(0,x) \equiv v_0=0$.


The eigenvalues are $\lambda=\{3,1\}$, and without too much difficulty we find

$$P = \textstyle \left[\begin{array}{cc}1&1\\1&-1\end{array}\right]$$

Hence, the solution is
\begin{align}
u(t,x) &= \frac{1}{2}\bigg[ u_0(x-3t) + u_0(x-t) \bigg], \\
v(t,x) &= \frac{1}{2}\bigg[ u_0(x-3t) - u_0(x-t) \bigg].
\end{align}

In [ ]:
using Plots
using LinearAlgebra
using LaTeXStrings
default(linewidth=3)
default(legendfontsize=12)


u0(x) = abs.(x) .<= 1

# Spatial grid
x = -2:0.1:12

ctr = 0
for t in 0:0.5:3
    # Compute u and v
    u = (u0(x .- 3t) .+ u0(x .- t)) ./ 2
    v = (u0(x .- 3t) .- u0(x .- t)) ./ 2

    # Create subplots
    p1 = plot(
        x, u,
        color = :blue,
        label = L"u(t,x)",
        ylim = (0, 1.05),
        xlim = (-2, 12),
        title = "t=$t"
    )

    p2 = plot(
        x, v,
        color = :red,
        label = L"v(t,x)",
        ylim = (-1.05/2, 1.05/2),
        xlim = (-2, 12)
    )

    p = plot(p1, p2, layout = (2, 1))
    display(p)


    ctr += 1

end


## 3. Hyperbolic Equations with Variable Coefficients  

What happens when the propagation speed is variable, e.g.,

$$
u_t + a(t,x)u_x = 0\,?
$$


In this example the solution is still constant along characteristics, but the characteristics are not straight lines.

Here, we get an ODE for the $x$-coordinate

$$
\frac{dx}{d\tau} = a(\tau,x), \quad x(0) = \xi.
$$


If, e.g., $a(\tau,x)=x$, then $x(\tau) = \xi e^{\tau}$ (so that $\xi = xe^{-t}$), and we get

$$
u(t,x) = \tilde{u}(\tau,\xi) = u_0(\xi) = u_0(xe^{-t}).
$$

### 3.1 Hyperbolic Systems with Variable Coefficients

We can extend the definition of hyperbolicity to systems:



:::{admonition} Definition 

A system of the form
$$
\mathbf{u}_t + A(t,x) \mathbf{u}_x + B(t,x) \mathbf{u} = F(t,x)
$$
is **hyperbolic** if there exists a matrix function $P(t,x)$ such that

$$
P(t,x) A(t,x) P^{-1}(t,x) = \textrm{diag}(\lambda_1(t,x),\dots,
\lambda_d(t,x)) = \Lambda(t,x)
$$

is diagonal with real eigenvalues and the matrix norms of $P(t,x)$
and $P^{-1}(t,x)$ are bounded in $x$ and $t$ for $x\in
\mathbb{R}$, $t\ge0$.
:::




## 4. Boundary Conditions 

Let's solve the 1D wave equation on a finite domain, e.g., $[0,1]$.

$$
u_t + a u_x =0, \quad 0\le x \le1, \ t\ge 0,
$$

We saw that if $a$ is positive then the information propagates to the _right_ and if $a$ is negative it propagates to the _left_.


<!-- <div style="display: flex; gap: 20px;">
  <img src="../img/IBVp_apos.svg" width="45%">
  <img src="../img/IBVp_aneg.svg" width="45%">
</div> -->

::::{grid} 
:gutter: 2

:::{grid-item-card}
:columns: 12 6 6 4
![IBVP a positive](../img/IBVp_apos.svg)
:::

:::{grid-item-card}
:columns: 12 6 6 4
![IBVP a negative](../img/IBVp_aneg.svg)
:::
::::

When $a>0$, in addition to the initial value $u(0,x)$ $0\le x \le 1$, we must also specify the boundary value $u(t,0)$ for all $t>0$, and when $a<0$ we must specify $u(t,1)$ for $t>0$.

<!-- <div style="display: flex; gap: 20px;">
  <img src="../img/IBVp_apos_spec.svg" width="45%">
  <img src="../img/IBVp_aneg_spec.svg" width="45%">
</div> -->

::::{grid} 1 2 
:outline:

:::{grid-item}
A
:::
:::{grid-item}
B
:::
::::

The problem of determining a solution when both initial and boundary data are present is known as an **Initial-Boundary Value Problem (IBVP)**.

Consider the hyperbolic system (assume $a>0$, $b>0$)

$$
  \left[\begin{array}{c}u\\v\end{array}\right]_t+
  \left[\begin{array}{cc}a&b\\b&a\end{array}\right]
  \left[\begin{array}{c}u\\v\end{array}\right]_x = 0
$$

on the interval $0\le x \le 1$.  The characteristic speeds are $(a+b)$ and $(a-b)$, so that with $w=u+v$, and $z=u-v$

$$
  \left[\begin{array}{c}w\\z\end{array}\right]_t+
  \left[\begin{array}{cc}a+b&\\&a-b\end{array}\right]
  \left[\begin{array}{c}w\\z\end{array}\right]_x = 0
$$

If $b<a$, then both characteristic speeds are positive, but when $b>a$, we get one positive and one negative speed.

<div style="display: flex; gap: 20px;">
  <img src="../img/IBVP_2same.svg" width="45%">
  <img src="../img/IBVP_2diff.svg" width="45%">
</div>

In the above figure, in the left panel $b<a$, so both characteristics propagate to the _right_. In the right panel $b>a$, so the characteristics propagate in opposite directions.

<div style="display: flex; gap: 20px;">
  <img src="../img/IBVP_2same_spec.svg" width="45%">
  <img src="../img/IBVP_2diff_spec.svg" width="45%">
</div>

In order for the IBVPs to be **well-posed** we must 
- [Left panel:] specify the initial condition and two boundary conditions at $x=0$; 
- [Right panel:] the initial condition, a boundary condition at $x=0$, and a boundary condition at $x=1$. 

Note that the specified boundary conditions must be linearly independent from the outgoing (leaving the domain) characteristic.

## 5. FD schemes for hyperbolic equations

### 5.1 FD schemes for the 1D wave equation

For the 1D one-way wave equation or transport equation

$$
u_t+au_x=0
$$

we can write down a number of finite difference approximations at $(t_n,x_m)$, e.g.,

\begin{align}
    \displaystyle \frac{v_{k}^{n+1} - v_{k}^{n}}{\Delta t} + a \frac{
      v_{k+1}^{n} - v_{k}^{n} }{\Delta x} = 0 && \hbox{FTFS}\\ 
    \displaystyle \frac{v_{k}^{n+1} - v_{k}^{n}}{\Delta t} + a \frac{
      v_{k}^{n} - v_{k-1}^{n} }{\Delta x} = 0 && \hbox{FTBS}\\
    \displaystyle \frac{v_{k}^{n+1} - v_{k}^{n}}{\Delta t} + a \frac{
      v_{k+1}^{n} - v_{k-1}^{n} }{ 2 \Delta x} = 0 && \hbox{FTCS}\\
    \displaystyle \frac{v_{k}^{n+1} - v_{k}^{n-1}}{2 \Delta t} + a \frac{
      v_{k+1}^{n} - v_{k-1}^{n} }{ 2 \Delta x} = 0 && \hbox{CTCS}, \textrm{(leapfrog)}\\
\end{align}

The schemes presented so far can all be written expressing $v_k^{n+1}$ as linear combinations of $v_{\mu}^{\nu}$ at previous time-levels $\nu \in \{n-1,\,n\}$.  

The FTFS scheme can be written as

$$
  v_k^{n+1} = (1+a r) v_{k}^{n} - a r v_{k+1}^{n}
$$

where $r = \Delta t/\Delta x$ is the ratio of the time- and space grid sizes. 

This scheme is a **one-step scheme** since it only involves information from one previous time-level.

The leapfrog scheme (CTCS) is a **two-step** (**multi-step**) scheme.

For $n$-step schemes, we need to allow for initialization of the first $n$ time-levels. Before we can apply an $n$-step scheme we must define $v_k^0$, $\dots$, $v_k^{n-1}$, $\forall k$.

#### 5.2 Example using Leapfrog

$$u_t+u_x=0$$

with initial condition

$$
  u_0(x) = \left\{ \begin{array}{ll}
      1-|x| & , \hbox{ if}\ |x|\le1 \\
      0 & , \hbox{ otherwise}
    \end{array}\right.
$$
  
and boundary condition

$$
  u(t,-2) = 0.
$$

Let's try with different $r = \Delta t/\Delta x$.


In [ ]:
# -------------------------------
# Shared FD utilities (plus `diff1_mat` from Module 1-7)
# -------------------------------

"""First-derivative differentiation matrix on a uniform grid.

- interior: 2nd-order centered
- boundaries: 1st-order one-sided
"""
# Dfferentiation matrix for a first-derivative
function diff1_mat(x)
    n = length(x)
    D = zeros(n, n)
    h = x[2] - x[1]
    D[1, 1:2] = [-1/h  1/h]              # Use a first-order forward difference at the left boundary
    for i in 2:n-1
        D[i, i-1:i+1] = [-1/2h  0  1/2h] # In the interior points, use a second-order centered difference
    end
    D[n, n-1:n] = [-1/h  1/h]            # Use a first-order backward difference at the right boundary
    D
end

"""Apply homogeneous Dirichlet BCs in-place."""
apply_dirichlet0!(u) = (u[1] = 0.0; u[end] = 0.0; u)

# -------------------------------
# Hyperbolic solvers as functions
# -------------------------------

"""Convenience: uniform grid spacing."""
Δx(x) = x[2] - x[1]

"""Leapfrog (CTCS) for linear advection u_t + a u_x = 0.

Returns (t, x, U) where U[n, i] ≈ u(t[n], x[i]).
"""
function leapfrog_advection(u0, a, x, T; r=0.8)
    h  = Δx(x)
    dt = r * h
    nt = round(Int, T/dt) + 1
    t  = range(0.0, step=dt, length=nt)

    D  = diff1_mat(x)        # re-used from Module 1-7
    A  = -a * D              # semi-discrete operator u_t = A*u

    U = zeros(nt, length(x))
    # first I.C. (zeroth-step)
    U[1, :] .= u0.(x)
    # first-step starter (exact shift is fine for this linear test)
    U[2, :] .= u0.(x .- a*dt)

    for n in 2:nt-1
        U[n+1, :] .= U[n-1, :] .+ 2dt .* (A * U[n, :])
        apply_dirichlet0!(U[n+1, :])
    end
    t, x, U
end

"""Lax–Friedrichs scheme for linear advection u_t + a u_x = 0.

"""
function lax_friedrichs_advection(u0, a, x, T; r=0.8)
    h  = Δx(x)
    dt = r * h
    nt = round(Int, T/dt) + 1
    t  = range(0.0, step=dt, length=nt)

    U = zeros(nt, length(x))
    U[1, :] .= u0.(x)

    for n in 1:nt-1
        U[n+1, 2:end-1] .= 0.5 .* U[n, 1:end-2] .+ 0.5 .* U[n, 3:end] .-
                                  (a*r/2) .* (U[n, 3:end] .- U[n, 1:end-2])
        apply_dirichlet0!(U[n+1, :])
    end
    t, x, U
end


In [ ]:
# Solution to the 1D wave equation (advection equation) u_t + a u_x = 0 using the Leapfrog (CTCS) scheme

# constant wave speed
a = 1.0
# final time
T = 2.0

# I.C., a triangle wave
u0(x) = (1 .- abs.(x)) .* (abs.(x) .<= 1)

# grid
x = collect(-2.0:0.1:4.0)

t, x, U = leapfrog_advection(u0, a, x, T; r=0.8)

animation = @animate for k in eachindex(t)
    plot(x, U[k, :];
        marker = :circle,
        label  = "leapfrog",
        xlims  = (-2, 4),
        ylims  = (-0.1, 1.1),
        grid   = true,
        title  = "t = $(round(t[k]; digits=3))",
    )
    plot!(x, u0.(x .- a*t[k]); label = "exact")
end

gif(animation, "leapfrog.gif", fps=10)


In [ ]:
# plot only the final time 
plot(x, U[end, :];
    marker = :circle,
    label  = "leapfrog",
    xlims  = (-2, 4),
    ylims  = (-0.1, 1.1),
    grid   = true,
    title  = "Final time T = $T",
)
plot!(x, u0.(x .- a*T); label = "exact")


When we try with $ r > 1$, clearly something "strange" happens

#### 5.3 Example using Lax-Friedrichs

$$
 \begin{array}{rll}
      \displaystyle \frac{v_{k}^{n+1} - v_{k}^{n-1}}{2\Delta t} + a \frac{
        v_{k+1}^{n} - v_{k-1}^{n} }{ 2\Delta x} = 0 && \hbox{
        Leapfrog}\\
      \displaystyle \frac{ v_k^{n+1} - \frac{1}{2}(v_{k+1}^n+v_{k-1}^n)}
      {\Delta t} + a \frac{v_{k+1}^n-v_{k-1}^n}{2\Delta x} = 0 && \hbox{
        Lax-Friedrichs}
    \end{array}
$$

Note that the Lax-Friedrichs scheme is a one-step scheme, whereas the leapfrog scheme is a 2-step scheme.

#### Example 8.2:
Same example as above:

$$u_t+u_x=0$$

with initial condition

$$
  u_0(x) = \left\{ \begin{array}{ll}
      1-|x| & , \hbox{ if}\ |x|\le1 \\
      0 & , \hbox{ otherwise}
    \end{array}\right.
$$
  
and boundary condition

$$
  u(t,-2) = 0.
$$




In [ ]:
# u_t + a u_x = 0 using the Lax–Friedrichs scheme

# constant wave speed
a = 1.0
# final time
T = 2.0

# I.C.
u0(x) = (1 .- abs.(x)) .* (abs.(x) .<= 1)

# store final-time solutions for each D
xs = Dict{Int, Vector{Float64}}()
ss = Dict{Int, Vector{Float64}}()

r = 0.8

for D in (5, 10, 20)
    Δx_D = 1.0 / D
    x  = collect(-2.0:Δx_D:4.0)

    t, x, U = lax_friedrichs_advection(u0, a, x, T; r=r)

    animation = @animate for k in eachindex(t)
        plot(x, U[k, :];
            marker = :circle,
            label  = "Lax–Friedrichs",
            xlims  = (-2, 4),
            ylims  = (-0.1, 1.1),
            grid   = true,
            title  = "D = $D,  t = $(round(t[k]; digits=3))",
        )
        plot!(x, u0.(x .- a*t[k]); label="exact")
    end
    gif(animation, "lax_friedrichs_D$(D).gif", fps=10)

    xs[D] = x
    ss[D] = vec(U[end, :])
end

# final comparison plot (at time T)
x5,  s5  = xs[5],  ss[5]
x10, s10 = xs[10], ss[10]
x20, s20 = xs[20], ss[20]

x_exact = x20
s_exact = u0.(x_exact .- a*T)

pfinal = plot(x5,  s5;  marker=:circle, label="Δx=1/5")
plot!(pfinal, x10, s10; marker=:diamond, label="Δx=1/10")
plot!(pfinal, x20, s20; marker=:x, label="Δx=1/20")
plot!(pfinal, x_exact, s_exact; label="Exact")

xlims!(pfinal, (0, 4))
ylims!(pfinal, (0, 1.1))
title!(pfinal, "Lax–Friedrichs r=$r (T=$T)")

display(pfinal)


- From the example above we can see that as $\Delta x \to 0$, the approximate solution gets better and better. 

- Consistency alone is necessary, but not sufficient for a Finite Difference Scheme (FDS) to be convergent (Recall Lax Equivalence theorem).

:::{admonition} Definition
A finite difference scheme $P_{k,h} v_{m}^{n}=0$ for a first-order equation is **stable** in a stability region $\Lambda$ if there is an integer $S$ such that for any positive time $T$, there is a constant $C_T$ such that

$$
    \Delta x \sum_{k=-\infty}^{\infty} \left| v_{k}^{n} \right|^2 \le
    C_T \Delta x \sum_{s=0}^{S} \sum_{k=-\infty}^{\infty} \left| v_{k}^{s}
    \right|^2
$$ 

for $0 \le n \Delta t \le T$, with $(\Delta t,\Delta x)\in\Lambda$.
:::

Recall that the quantity

$$
  \|w\|_h = \left[ h \sum_{k=-\infty}^{\infty} \left| w_{k} \right|^2
  \right]^{1/2}
$$

is the $L^2$ norm of the grid function $w$, and is a measure of the size (energy) of the solution. The multiplication by $h\equiv \Delta x$ is needed so that the norm is not sensitive to grid refinements (the number of points increase as $h\to 0$).

With this notation, the inequality in the definition can be written

$$
\|v^n\|_h \le \left[ C_T \sum_{s=0}^{S}\|v^s\|^2_h \right]^{1/2}
  \quad\Leftrightarrow\quad
  \|v^n\|_h \le C_T^* \sum_{s=0}^{S}\|v^s\|_h
$$

- The inequality expresses a limit (in terms of energy) of how much the solution can grow. 
- Typically $S=(n-1)$ for an $n$-step scheme.



## 6. Stability of hyperbolic PDEs  

We now turn our attention to the key stability criterion for hyperbolic PDEs.

In the example above, we saw some numerical evidence of the leapfrog scheme (applied to $u_t+au_x=0$, $a=1$) breaking down when $ r>1$.

The condition $|a r|<1$ is **necessary** for stability of many _explicit_ FDS.

- Consistency implies that the solution of the PDE, _if it is smooth_, is an approximate solution of the finite difference scheme (FDS).

- Convergence means that a solution of the FDS approximates a solution of the PDE.

- Consistency is **necessary**, but **not sufficient** for a FDS to be convergent.

We illustrate this with an example.

### Consistency $\not\Rightarrow$ Convergence

We consider the one-way wave equation with constant $a=1$ propagation speed, and apply the forward-time forward-space (F.T.F.S.) scheme

$$
\frac{ u_{k}^{n+1} - u_{k}^{n} } {\Delta t} + \frac{u_{k+1}^n-u_{k}^{n}} {\Delta x}  = 0.
$$

We rewrite this:

$$
u_{k}^{n+1} = u_{k}^{n} - \underbrace{\frac{\Delta t}{\Delta x}}_{r} u_{k+1}^n - u_{k}^n = (1 + r) u_k^n - r u_{k+1}^n
$$

Let the initial condition be given by:

$$
  u_0(x) = \left\{ \begin{array}{ll}
      1 & \hbox{if}\ -1\le x \le 0,\\
      0 & \hbox{elsewhere}
    \end{array}\right.
$$

Hence the exact solution is $u(t,x) = u_0(x-t)$, i.e., a "hump" of height and width one, traveling to the right with unitary speed.

The IC for the FDS is discretized as:

$$
  v_k^0 = \left\{ \begin{array}{ll}
      1 & \hbox{if}\ -1\le k \Delta x \le 0,\\
      0 & \hbox{elsewhere}
    \end{array}\right.
$$


Let us see what **region** the numerical scheme depends on.

![Representation of the region of dependence for a FTFS scheme](../img/exact_soln.svg)

The figure above illustrates how the exact solution propagates; it is one in the pink band/region, and zero outside the band. 

The IC of the FD scheme says that the values are zeros everywhere, except at the four points: $u_0^0$, $u_{-1}^0$, $u_{-2}^0$, and $u_{-3}^0$, where it is $1$.

![Representation of how the FTFS scheme propagates](../img/fds_soln.svg)

The figure above illustrates how the FD scheme solution propagates. In particular we have that $v_k^n\equiv0$, $\forall k>0,\ n\ge0$. Hence, $v_k^n \not\rightarrow u(t^n,x_k)$ for $(t^n,x_k)$ in the part of the band strictly in the right half plane - $u$ is one there, but $v_k^n$ is zero, no matter how much we refine the grid.


If a scheme is _convergent_, then as $v_k^n$ converges to $u(t,x)$, then  $v_k^n$ is bounded in some sense; this is the essence of _stability_.

- For almost all schemes there are restrictions on the way $\Delta x$ and $\Delta t$ can be chosen so that the particular scheme is stable. 

:::{admonition} Definition: Stability region
A **stability region** is any bounded non-empty region of the first quadrant of $\mathbb{R}^2$ that has the origin as an accumulation point.
:::

![Representation of the stability region](../img/stability_region.svg)



- An **explicit** scheme is a scheme that can be written as

$$
v_k^{n+1} = \sum^{\hbox{\tiny finite}}_{n'\le n} v_{k'}^{n'}
$$

- Implicit schemes, where the sum may contain terms with $n'=n+1$ on the RHS, will be discussed in the future.



### 6.1 The Courant-Friedrichs-Lewy Condition

The following result covers all one-step schemes we have seen so far.

:::{admonition} Theorem

For an explicit scheme for the hyperbolic equation

$$
u_t + a u_x = 0
$$

of the form

$$
 v_k^{n+1} = \alpha v_{k+1}^n + \beta v_k^n + \gamma v_{k-1}^n
$$

with $r = \Delta t/\Delta x$ held constant, a _necessary_ condition for _stability_ is the **Courant-Friedrichs-Lewy (CFL)condition**,

$$
    |a r|\le 1.
$$

For systems of equations for which $\mathbf{v}$ is a vector and $\alpha$, $\beta$, and $\gamma$ are matrices, we must have
$|a_i r|\le1$ for all eigenvalues $a_i$ of the matrix $A$.
:::

![Representation of the region of dependence and CFL condition](../img/CFL.svg)

In the above figure, we see an illustration of the CFL condition, with $ r=1$ held fixed. 

The green triangle shows the **region of dependence**, i.e., what region influences $v^{n+1}_k$ (actually only the three points at the base of the triangle contribute). 

The blue arrow corresponds to a characteristic with speed $a=0.75$, which carries information _inside_ the region of dependence.

The red arrow, corresponding to a characteristic speed of $a=1.5$ carries information from _outside_ the region of dependence; this information _cannot_ be captured by the scheme.

Courant, Friedrichs and Lewy also proved the following theorem.

:::{admonition} Theorem
There are no _explicit_, _unconditionally stable_, consistent finite difference schemes for hyperbolic systems of partial differential equations.
:::

One way of thinking about these theorems is to define the **numerical speed of propagation** as $ r^{-1} = \Delta x/\Delta t$, and note that a *necessary* condition for the stability of a scheme is

$$
r^{-1} \ge |a|.
$$

This guarantees that the FDS can propagate information (energy) at least as fast as the PDE.

$r^{-1}$ is the "speed limit" on the grid; which explains why (with $a=1$), we saw the breakdown of the leapfrog scheme when $r>1 \Leftrightarrow r^{-1} < 1$.

## 7. Conditional and unconditional stability

In general, even for other classes of problems (not necessarily hyperbolic PDEs) in the mathematics literature, we can find a variety of definitions of stability.

Let's consider a two-level difference scheme in matrix form:

$$
\mathbf{u}^{n+1} = Q \mathbf{u}^{n} , n \geq 0
$$

which will generally be a difference scheme for solving an initial-value problem (IVP) on $\mathbb{R}$, for a homogeneous linear PDE.

- A _stronger_ definition of **stability** is that 

:::{admonition} Definition 
The difference scheme $\mathbf{u}^{n+1} = Q \mathbf{u}^{n} , n \geq 0$ is said to be **stable** with respect to the norm $\| \cdot \|$ if there exists positive constants $\Delta x_0$ and $\Delta t_0$, and non-negative constants $K$ and $\beta$ such that

$$
\| \mathbf{u}^{n+1}  \| \leq K e^{\beta t} \| \mathbf{u}^0 \|,
$$

for $0 \leq t = (n+1) \Delta t$, $0 < \Delta x \leq \Delta x_0$ and $0 < \Delta t \leq \Delta t_0$.
:::

**Remark**:
- With this defintion, the solution is allowed to grow with time, but not with the number of _time steps_.

Another common definition that is used is one that does _not_ allow for exponential growth, as in:

$$
\| \mathbf{u}^{n+1}  \| \leq K  \| \mathbf{u}^0 \|.
$$

This latter definition of stability implies that the solutions to the difference equation must be _bounded_.

:::{admonition} Proposition
The difference scheme $\mathbf{u}^{n+1} = Q \mathbf{u}^{n} , n \geq 0$ is stable with respect to the $\| \cdot \|$ norm _if and only if_ there exists positive constants $\Delta x_0$ and $\Delta t_0$ and non-negative constants $K$ and $\beta$ such that

$$
\| Q^{n+1} \| \leq K e^{\beta t}
$$

for $0 \leq t = (n+1) \Delta t$, $0 < \Delta x \leq \Delta x_0$, and $0 < \Delta t \leq \Delta t_0$.

:::


**Remarks**:

- We notet that in the Preposition above the norm $\| Q^{n+1} \|$ is a matrix norm, which is induced by the norm defined on the space in which we are working.

- As with convergence, it is difficult to prove stability directly. (We will develop a method to find a necessary condition later in the course).

- Every time we can find a condition (such as the $ |a r|\le 1$ condition above in the CFL condition), we say that the scheme is **conditionally stable**.
- In the case where no restrictions on the relationship between $\Delta t$ and $\Delta x$ are needed for stability, we say that the scheme is **stable** or **unconditionally stable**.